# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [2]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [3]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [5]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Create two datasets
* one with digits below 5
* one with 5 and above

In [7]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will be used for transfer learning
* Freeze these layers during fine-tuning process

In [8]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [10]:
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [11]:
model = Sequential(feature_layers + classification_layers)

Check the model summary

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [13]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 47s 200ms/step - loss: 1.5915 - accuracy: 0.2684 - val_loss: 1.5731 - val_accuracy: 0.3649
Epoch 2/5
230/230 [==============================] - 46s 201ms/step - loss: 1.5685 - accuracy: 0.3279 - val_loss: 1.5455 - val_accuracy: 0.4779
Epoch 3/5
230/230 [==============================] - 47s 204ms/step - loss: 1.5435 - accuracy: 0.3952 - val_loss: 1.5151 - val_accuracy: 0.6276
Epoch 4/5
230/230 [==============================] - 46s 199ms/step - loss: 1.5131 - accuracy: 0.4609 - val_loss: 1.4804 - val_accuracy: 0.7227
Epoch 5/5
230/230 [==============================] - 47s 203ms/step - loss: 1.4806 - accuracy: 0.5131 - val_loss: 1.4413 - val_accuracy: 0.7667
Training time: 0:04:23.307833
Test score: 1.4412593841552734
Test accuracy: 0.7667146921157837


Freeze only the feature layers

In [14]:
for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [16]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 26s 102ms/step - loss: 1.5814 - accuracy: 0.2676 - val_loss: 1.5472 - val_accuracy: 0.3929
Epoch 2/5
240/240 [==============================] - 18s 75ms/step - loss: 1.5372 - accuracy: 0.3596 - val_loss: 1.5006 - val_accuracy: 0.5046
Epoch 3/5
240/240 [==============================] - 19s 78ms/step - loss: 1.4965 - accuracy: 0.4261 - val_loss: 1.4557 - val_accuracy: 0.6042
Epoch 4/5
240/240 [==============================] - 16s 69ms/step - loss: 1.4537 - accuracy: 0.4936 - val_loss: 1.4122 - val_accuracy: 0.6885
Epoch 5/5
240/240 [==============================] - 19s 78ms/step - loss: 1.4141 - accuracy: 0.5475 - val_loss: 1.3697 - val_accuracy: 0.7463
Training time: 0:01:38.211552
Test score: 1.369651436805725
Test accuracy: 0.7462541460990906


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [20]:
# I reused the feature_layers and classification_layers
## which were first used earlier in the procedure.

feature_layers_sa = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [21]:
classification_layers_sa = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [22]:
model_sa = Sequential(feature_layers_sa + classification_layers_sa)

In [23]:
model_sa.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_6 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [24]:
train_model(model_sa,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 61s 250ms/step - loss: 1.5867 - accuracy: 0.2669 - val_loss: 1.5620 - val_accuracy: 0.3466
Epoch 2/5
240/240 [==============================] - 50s 210ms/step - loss: 1.5504 - accuracy: 0.3554 - val_loss: 1.5204 - val_accuracy: 0.5001
Epoch 3/5
240/240 [==============================] - 50s 209ms/step - loss: 1.5111 - accuracy: 0.4456 - val_loss: 1.4740 - val_accuracy: 0.6135
Epoch 4/5
240/240 [==============================] - 51s 214ms/step - loss: 1.4655 - accuracy: 0.5192 - val_loss: 1.4201 - val_accuracy: 0.7077
Epoch 5/5
240/240 [==============================] - 49s 204ms/step - loss: 1.4133 - accuracy: 0.5922 - val_loss: 1.3576 - val_accuracy: 0.7786
Training time: 0:04:21.639510
Test score: 1.3575791120529175
Test accuracy: 0.7785561680793762


In [25]:
# Just like in the procedure, after training the model for the
## first 5 digits (0-4) we will freeze only the feature layers.

for l in feature_layers_sa:
    l.trainable = False

In [26]:
model_sa.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_6 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [27]:
train_model(model_sa,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 19s 75ms/step - loss: 1.3635 - accuracy: 0.6440 - val_loss: 1.3148 - val_accuracy: 0.8208
Epoch 2/5
240/240 [==============================] - 18s 76ms/step - loss: 1.3237 - accuracy: 0.6951 - val_loss: 1.2722 - val_accuracy: 0.8504
Epoch 3/5
240/240 [==============================] - 17s 70ms/step - loss: 1.2859 - accuracy: 0.7307 - val_loss: 1.2298 - val_accuracy: 0.8727
Epoch 4/5
240/240 [==============================] - 18s 74ms/step - loss: 1.2450 - accuracy: 0.7633 - val_loss: 1.1885 - val_accuracy: 0.8922
Epoch 5/5
240/240 [==============================] - 18s 74ms/step - loss: 1.2048 - accuracy: 0.7914 - val_loss: 1.1477 - val_accuracy: 0.9054
Training time: 0:02:22.801219
Test score: 1.1477142572402954
Test accuracy: 0.9054290652275085


The model trained in the supplementary activity has a higher test score and accuracy compared to the first model trained in the procedure.

### Conclusion
- In this activity, we were taught on how to do transfer learning, which in simpler terms, is the utilization of pretrained models to be reused on new problems. The model trained in this activity was applied to the CIFAR-10 dataset, using the digits from 0-9. This activity helped us better understand different use-cases and applications of deep learning in training models.
